# Notes on Chapters 1 - 3 in Sutton RL book

## Chapter 1

The following is my summary and paraphrasing of Chapter 1 in Sutton and Barto's Reinforcement Learning: An Introduction

### 1.1

"Reinforcement learning is learning what to do - how to map situations to actions - so as to maximize a numerical reward signal." For tic-tac-toe this would be something like learning how to take actions to maximize a win rate.

"A learning agent must be able to sense the state of its environment to some extent and must be able to take some actions to affect the state." In terms of tic-tac-toe this means the agent needs to be able to observe the state of the board and it needs to be able to place Xs and Os where it chooses.

"The agent must have a goal or goals relating to the state of the environment"; here the goal would be to win.

"Exploration vs Exploitation - To get a lot of reward, an agent has to use actions it has used in the past that it knows produces reward, but to find reward giving actions, an agent must try actions its never tried before. So the agent has to try a variety of actions and progressively favor those that appear best." In tic-tac-toe this could be equivalent to continually trying new moves in hopes of uncovering a better strategy than what has already been found.

### 1.3

4 main subelements of a reinforcement learning system:
    - Policy
    - Reward signal
    - Value function
    - Environment model (optional)

"Policy describes the learning agent's way of behaving at a given time, it provides a mapping from the state of the environment to what action to take in that state." In tic-tac-toe, the policy would describe what move to make depending on the state of the board.

"The reward signal defines the goal in an RL problem." At each time step, the environment sends a reward value to the agent. The agent's goal is to maximize the reward it receives over long period of time. In tic-tac-toe, the reward could come from winning, so the agent would want to make moves that are winning moves so that it can receive more reward.

"Since the reward signal operates at each time step, it is a short-term value; the value function declares what is good in the long run." The value of a state is the total reward an agent can expect to gather over the future, starting with that state. Values indicate the long-term desirability of states, taking into account the states that are likely to follow, and the available reward in those states. In tic-tac-toe, a high value move might be one that likely leads to a winning sequence of moves.

"When we are making action choices, we use values as judgment, not rewards." We want actions that yield the highest value, not the highest reward, because these are the best actions over the long run. It's much harder to determine values than rewards; rewards are given to us by the environment, while values must be estimated using an agent's past experiences and observations. 

"The model of the environment mimics the environment, or it allows inferences to be made about how the environment will behave." i.e. given a state and action, a model might be able to predict the next state and reward. We use models for planning. In tic-tac-toe, this might mean a model of the board and the other player where a move can be made in the model and the model tries to return a prediction of the opponents responding move and the reward from that next state.

### 1.5

Tic-tac-toe example. 

Let's consider draws and losses to be equally bad. How can we build a player that will find the imperfections in its opponent's play and learn to maximize its chances of winning?

This can't be solved readily through classical techniques. The minimax solution from game theory doesn't work because it asssumes a particular way of playing by the opponent, and something like dynamic programming requires as input complete specifications of the opponent, including the probabilities of each move the opponent could make in each board state. 

We could play lots of games against the opponent and learn some model of the opponents behavior, and then apply dynamic programming to those estimated move probabilities from the model to compute the optimal moves.

An evolutionary method would search the space of possible policies for one with a high probability of winning against the opponent. In this case, the policy is a rule that tells the player what move to make for every state of the game.

Tic-tac-toe with method making use of a value function:

We set up a table of numbers, one for each state of the game. Each number is the latest estimate of the probability of winning from that state. This estimate is considered the state's value, and the whole table is the learned value function. 

If we are playing X's, then each state where there are 3 X's in a row has a probability of 1, since we've already won, and each state where there are 3 O's in a row or the board is full has a probability of 0, since we've either lost or drawn and we can't win from that state. Make the initial values of all other states .5, to guess an initial 50% probability of winning.

Play lots of games against the opponent. To pick our next move we look at the states that would result from each possible move and check the corresponding win probability in the table. Most often we move greedily, picking the move that leads to the state with the highest value. But sometimes we randomly select a move from the table, and these are our exploratory moves, which allow us to see states that we might otherwise never find. 

While playing, we change the values of the states we wind up in. We try to make them more accurate estimates of the probability of winning. The current value of the earlier state is updated to be closer to the value of the later state. This is done by moving the value of the earlier state a fraction of the way towards the value of the later state.
    - Let s be the state before the greedy move
    - s' is the state after the greedy move
    - V(s) is the update to the estimated value of s
    - alpha is learning rate
    
V(s) <-- V(s) + alpha * [V(s') - V(s)]

This update rule is an example of temporal difference learning, because its changes are based on a difference at two different times (V(s') - V(s)).

If alpha is reduced to zero over time, this method converges for any fixed player to the true probabilities of winning for any given state, given optimal play by the method. If alpha isn't reduced all the way to zero over time, then this method also plays well against players that slowly change their style of playing.

## Tic-tac-toe code

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

## TODO:

- Fix gen_o_values function::DONE
- Start making changes to make more general for n by n tic-tac-toe
- Clean up code 

In [2]:
class tictactoe:
    def __init__(self):
        self.x_win = False
        self.o_win = False
        self.game_over = False
        self.v = dict()
        self.chars = [0, 1, 2]
        
    def gen_state_arr(self):
        arr = []
        for i in range(3):
            for j in range(3):
                for k in range(3):
                    for t in range(3):
                        for u in range(3):
                            for s in range(3):
                                for q in range(3):
                                    for y in range(3):
                                        for z in range(3):
                                            tmp = np.array([self.chars[i], self.chars[j], self.chars[k], self.chars[t], self.chars[u], self.chars[s],
                                                       self.chars[q], self.chars[y], self.chars[z]]).reshape(3, 3)
                                            if self.check_legal(tmp) == True:
                                                arr.append(tmp)
                                                
        return np.asarray(arr)
        
    def gen_x_values(self, state_arr):
        vals = []
        for i in range(len(state_arr)):
            value = self.check_over(state_arr[i])
            vals.append(value)
        vals = np.asarray(vals)
        vals = vals[vals < 3]
        return vals
    
    def gen_o_values(self, x_values):
        return 1 - x_values
    
    def check_legal(self, state):
        x_count, o_count, empty_count = 0, 0, 0
        state_cpy = np.copy(state).ravel()
        for i in range(len(state_cpy)):
            if state_cpy[i] == 1: x_count += 1
            elif state_cpy[i] == 2: o_count += 1
            elif state_cpy[i] == 0: empty_count += 1
        
        if x_count == o_count + 1 or x_count == o_count:
            # check for X and O win
            if self.check_over(state) == 3:
                return False
            # check for O win and num Xs = num Os
            if x_count != o_count and self.check_over(state) == 0:
                return False
            # check for X win and num Xs = num Os + 1
            if self.check_over(state) == 1 and x_count != o_count + 1:
                return False
            return True
        return False

        
    def check_over(self, state):
        x_vec = np.ones(3, dtype=int)
        o_vec = np.full(3, 2, dtype=int)
        x_win, o_win = 0, 0
        
        state_diag = np.diag(state).astype(int)
        state_rev_diag = np.fliplr(state).diagonal().astype(int)
        
        if np.array_equal(state_diag, x_vec) or np.array_equal(state_rev_diag, x_vec):
            x_win += 1
        
        if np.array_equal(state_diag, o_vec) or np.array_equal(state_rev_diag, o_vec):
            o_win += 1
        
        cols = [state[:, 0].astype(int), state[:, 1].astype(int), state[:, 2].astype(int)]
        rows = [state[0, :].astype(int), state[1, :].astype(int), state[2, :].astype(int)]
        
        for i in cols:
            if np.array_equal(i, x_vec):
                x_win += 1
            elif np.array_equal(i, o_vec):
                o_win += 1
        
        for i in rows:
            if np.array_equal(i, x_vec):
                x_win += 1
            elif np.array_equal(i, o_vec):
                o_win += 1
        
        if x_win >= 1 and o_win == 1:
            return 3
        elif x_win >= 1:
            return 1
        elif o_win == 1:
            return 0
        else:
            return 0.5
    
    def human_viz(self, state):
        tmp_board = np.empty(state.shape, dtype=str)
        for i in range(len(state)):
            for j in range(len(state[i])):
                if state[i][j] == 1:
                    tmp_board[i][j] = 'X'
                elif state[i][j] == 2:
                    tmp_board[i][j] = 'O'
                elif state[i][j] == 0:
                    tmp_board[i][j] = '-'
        print(tmp_board)
        
    def find_next_move_slow(self, state_arr, state, player):
        tmp_states = []
        inds = []
        if player is 1:
            for i in range(len(state)):
                for j in range(len(state[i])):
                    tmp = np.copy(state)
                    if tmp[i][j] == 0:
                        tmp[i][j] = 1
                        tmp_states.append(tmp)
            
            for i in range(len(tmp_states)):
                for j in range(len(state_arr)):
                    if np.array_equal(tmp_states[i], state_arr[j]):
                        inds.append(j)
            return inds
        
        if player is 2:
            for i in range(len(state)):
                for j in range(len(state[i])):
                    tmp = np.copy(state)
                    if tmp[i][j] == 0:
                        tmp[i][j] = 2
                        tmp_states.append(tmp)
                        
            for i in range(len(tmp_states)):
                for j in range(len(state_arr)):
                    if np.array_equal(tmp_states[i], state_arr[j]):
                        inds.append(j)
            return inds
    
    def find_next_move(self, state_arr, state, player):
        tmp_states = []
        inds = []
        sort_inds = np.asarray(np.where(state == 0))
        if player is 1:
            for i in range(sort_inds.shape[1]):
                tmp = np.copy(state)
                tmp_ind = sort_inds[:, i]
                tmp[tmp_ind[0], tmp_ind[1]] = 1
                inds.append(np.argmax((tmp == state_arr).sum(axis=1).sum(axis=1)))
            return inds
        
        if player is 2:
            for i in range(sort_inds.shape[1]):
                tmp = np.copy(state)
                tmp_ind = sort_inds[:, i]
                tmp[tmp_ind[0], tmp_ind[1]] = 2
                inds.append(np.argmax((tmp == state_arr).sum(axis=1).sum(axis=1)))
            return inds
        
    def get_current_state_index(self, state, state_arr):
        for i in range(len(state_arr)):
            if np.array_equal(state_arr[i], state):
                return state, i
        return 'Current state invalid.'
    
    def human_play(self, state_arr, agent, agent_char):
        state = state_arr[0]
        iter_count = 0
        if agent_char is 1:
            while self.check_over(state) == 0.5 and iter_count <= 8:
                cur_state_ind = self.get_current_state_index(state, state_arr)
                if iter_count % 2 == 0:
                    move_inds = self.find_next_move(state_arr, state, 1)
                    new_move_ind = agent.exploit_learned_probs(move_inds, cur_state_ind[1], .1)
                    state = state_arr[new_move_ind]
                else:
                    self.human_viz(state)
                    move_valid = False
                    while move_valid == False:
                        human_move = input('Input your move coordinates, separated by a comma: ')
                        coord_1, coord_2 = int(human_move[0]), int(human_move[-1])
                        if state[coord_1, coord_2] != 0:
                            print('Invalid move, try again.')
                            continue
                        tmp_state = np.copy(state)
                        tmp_state[coord_1, coord_2] = 2
                        if self.check_legal(tmp_state) == True:
                            state = tmp_state
                            move_valid = True
                iter_count += 1

        if agent_char is 2:
            while self.check_over(state) == 0.5 and iter_count <= 8:
                cur_state_ind = self.get_current_state_index(state, state_arr)
                if iter_count % 2 != 0:
                    move_inds = self.find_next_move(state_arr, state, 2)
                    new_move_ind = agent.exploit_learned_probs(move_inds, cur_state_ind[1], .1)
                    state = state_arr[new_move_ind]
                else:
                    self.human_viz(state)
                    move_valid = False
                    while move_valid == False:
                        human_move = input('Input your move coordinates, separated by a comma: ')
                        coord_1, coord_2 = int(human_move[0]), int(human_move[-1])
                        if state[coord_1, coord_2] != 0:
                            print('Invalid move, try again.')
                            continue
                        tmp_state = np.copy(state)
                        tmp_state[coord_1, coord_2] = 1
                        if self.check_legal(tmp_state) == True:
                            state = tmp_state
                            move_valid = True
                iter_count += 1

        self.human_viz(state)


In [3]:
ttt = tictactoe()

In [4]:
arr = ttt.gen_state_arr()

In [11]:
class td_agent:
    def __init__(self, values, p_random_move, alpha_start):
        self.vals = values
        self.p_random_move = p_random_move
        self.alpha = alpha_start
    
    def alpha_anneal(self, alpha_start, alpha_end, num_iters):
        self.alpha -= (alpha_start - alpha_end)/num_iters 
    
    def get_move_update_values(self, move_inds, current_state_ind):
        if np.random.binomial(1, self.p_random_move) == 1:
            move_ind = np.random.randint(0, len(move_inds))
            move = move_inds[move_ind]
        else:
            greedy_move = np.argmax(self.vals[move_inds])
            move = move_inds[greedy_move]
            self.vals[current_state_ind] += self.alpha * (self.vals[move] - self.vals[current_state_ind])
        
        return move
    
    def exploit_learned_probs(self, move_inds, current_state_ind, learn=False):
        move = move_inds[np.argmax(self.vals[move_inds])]
        if learn == True:
            self.vals[current_state_ind] += self.alpha * (self.vals[move] - self.vals[current_state_ind])
        return move

In [13]:
game = tictactoe()
state_arr = game.gen_state_arr()

x_vals = np.load('/home/jfp15b/Documents/reinforcement-learning/tictactoe_learned_vals/x_game_vals_fixed2.npy')
o_vals = game.gen_o_values(x_vals)

TD_x = td_agent(x_vals, .2, .4)
TD_o = td_agent(o_vals, .2, .4)

In [9]:
ts1 = state_arr[0]
# win probability should be 1
ts2 = np.array([1, 0, 1, 2, 2, 0, 1, 0, 2]).reshape(3, 3)
ts3 = np.array([1, 0, 1, 2, 1, 2, 0, 2, 0]).reshape(3, 3)
ts4 = np.array([1, 2, 1, 2, 0, 2, 1, 2, 1]).reshape(3, 3)
ts5 = np.array([2, 0, 0, 2, 1, 0, 1, 1, 2]).reshape(3, 3)
# win probability should be 0
ts6 = np.array([2, 0, 2, 0, 1, 1, 2, 1, 1]).reshape(3, 3)
ts7 = np.array([2, 2, 0, 1, 0, 0, 1, 0, 1]).reshape(3, 3)
ts8 = np.array([2, 0, 2, 1, 2, 1, 1, 1, 0]).reshape(3, 3)
ts9 = np.array([1, 1, 0, 1, 2, 0, 2, 0, 0]).reshape(3, 3)
ts10 = np.array([1, 1, 0, 0, 1, 0, 2, 2, 0]).reshape(3, 3)

ts_inds = []
ts_ = [ts1, ts2, ts3, ts4, ts5, ts6, ts7, ts8, ts9, ts10]
for i in ts_:
    ts_inds.append(game.get_current_state_index(i, state_arr)[1])
    
print(TD_x.vals[ts_inds])

[0.56139867 0.5        0.67195    0.5        0.5        0.5
 0.5        0.45       0.455      0.3645    ]


In [15]:
x_wins, o_wins, draws = 0, 0, 0
x_win_states, o_win_states, draw_states = [], [], []    
for i in range(5000):
    iter_count = 0
    cur_state = state_arr[0]
    while game.check_over(cur_state) == 0.5 and iter_count <= 8:
        cur_state_ind = game.get_current_state_index(cur_state, state_arr)
        if iter_count % 2 == 0:
            move_inds = game.find_next_move(state_arr, cur_state, 1)
            new_move_ind = TD_x.get_move_update_values(move_inds, cur_state_ind[1])
            cur_state = state_arr[new_move_ind]
        else:
            move_inds = game.find_next_move(state_arr, cur_state, 2)
            new_move_ind = TD_o.exploit_learned_probs(move_inds, cur_state_ind[1])
            cur_state = state_arr[new_move_ind]
        iter_count += 1
    if game.check_over(cur_state) == 1:
        x_wins += 1
        x_win_states.append(cur_state)
    if game.check_over(cur_state) == 0:
        o_wins += 1
        o_win_states.append(cur_state)
    if game.check_over(cur_state) == .5:
        draws += 1
        draw_states.append(cur_state)
    TD_x.alpha_anneal(.4, .1, 5000)
    if i%1000 == 0:
        print('Iter:', i)
        print('X values for test states:', TD_x.vals[ts_inds])
        print('O values for test states:', TD_o.vals[ts_inds])
        print()
        np.save('x_game_vals_alpha_anneal.npy', TD_x.vals)
        np.save('o_game_vals_alpha_anneal.npy', TD_o.vals)
        TD_o.vals = game.gen_o_values(TD_x.vals)

Iter: 0
X values for test states: [0.56139867 0.5        0.67195    0.5        0.5        0.5
 0.5        0.45       0.455      0.3645    ]
O values for test states: [0.43860133 0.5        0.32805    0.5        0.5        0.5
 0.5        0.55       0.545      0.6355    ]



KeyboardInterrupt: 

In [27]:
game.human_play(state_arr, TD_o, 2)

[['-' '-' '-']
 ['-' '-' '-']
 ['-' '-' '-']]


Input your move coordinates, separated by a comma:  22


[['O' '-' '-']
 ['-' '-' '-']
 ['-' '-' 'X']]


Input your move coordinates, separated by a comma:  11


[['O' '-' '-']
 ['O' 'X' '-']
 ['-' '-' 'X']]


Input your move coordinates, separated by a comma:  20


[['O' '-' '-']
 ['O' 'X' 'O']
 ['X' '-' 'X']]


Input your move coordinates, separated by a comma:  21


[['O' '-' '-']
 ['O' 'X' 'O']
 ['X' 'X' 'X']]


In [28]:
game.human_play(state_arr, TD_x, 1)

[['-' '-' '-']
 ['-' 'X' '-']
 ['-' '-' '-']]


Input your move coordinates, separated by a comma:  02


[['-' 'X' 'O']
 ['-' 'X' '-']
 ['-' '-' '-']]


Input your move coordinates, separated by a comma:  21


[['-' 'X' 'O']
 ['-' 'X' 'X']
 ['-' 'O' '-']]


Input your move coordinates, separated by a comma:  10


[['X' 'X' 'O']
 ['O' 'X' 'X']
 ['-' 'O' '-']]


Input your move coordinates, separated by a comma:  22


[['X' 'X' 'O']
 ['O' 'X' 'X']
 ['X' 'O' 'O']]


In [18]:
best_x_vals, best_o_vals = np.load('x_game_vals_fixed.npy'), np.load('o_game_vals_fixed.npy')

In [24]:
best_x_agent = td_agent(best_x_vals, .2)
best_o_agent = td_agent(best_o_vals, .2)

In [23]:
game.human_play(state_arr, best_x_agent, 1)

NameError: name 'best_x_agent' is not defined

In [28]:
game.human_play(state_arr, best_o_agent, 2)

[['-' '-' '-']
 ['-' '-' '-']
 ['-' '-' '-']]


Input your move coordinates, separated by a comma:  0,0


[['X' '-' '-']
 ['O' '-' '-']
 ['-' '-' '-']]


Input your move coordinates, separated by a comma:  2,2


[['X' 'O' '-']
 ['O' '-' '-']
 ['-' '-' 'X']]


Input your move coordinates, separated by a comma:  0,2


[['X' 'O' 'X']
 ['O' 'O' '-']
 ['-' '-' 'X']]


Input your move coordinates, separated by a comma:  2,0


[['X' 'O' 'X']
 ['O' 'O' 'O']
 ['X' '-' 'X']]


In [119]:
print(x_wins, o_wins, draws)

14731 2929 27340


## Chapter 2

The following is my notes, summary, and paraphrasing of Chapter 2 in Sutton and Barto's Reinforcement Learning: An Introduction

### Multi-Armed Bandits

"The most important feature distinguishing reinforcement learning from other types of learning is that it uses training information that $evaluates$ the actions taken rather than $instructs$ by giving the correct actions."

If we used feedback that is only evaluating each move, this will tell us how good the action taken was, but not if it was the best or worst action we could've possibly taken.

Using feedback that is just instructive will only tell us the right action to take, regardless of the action that we took.

### 2.1: A $k$-armed Bandit Problem

The problem:

We repeatedly have to choose from among $k$ different actions. After each action, a reward is given, the reward is picked from an unchanging probability distribution that depends on the action chosen. The goal is to maximize the (expected) total reward over an arbitrary time period.

"This is the original form of the k-armed bandit problem, so named by analogy to a slot machine, or "one armed bandit", except that it has $k$ levers instead of one. Each action selection is like a play of one of the slot machine's levers, and the rewards are payoffs for hitting the jackpot. Through repeated action selections you are to maximize your chances of winning by concentrating your actions on the best levers."



In the k-armed bandit problem, each action we could choose has an expected or average reward given that action is chosen, we call this the $value$ of that action. 

Notation:
    - Action selected on time step t is A<sub>t<sub/>
    - Reward at time step t is R<sub>t<sub/>
    
So, the value of an action $a$, denoted as $q$<sub>*</sub> $(a)$ is our expected reward given that $a$ is picked.
    
$q$<sub>*</sub>($a$) = E[$R$<sub>t</sub> | $A$<sub>t</sub> = $a$]
    
If we knew the values of each action with certainty, it would be easy to solve the k-armed bandit problem, because we would just choose the action with the highest value at each time step.
We assume to not know the action values with certainty. At best we may have estimates. The estimated value of action $a$ at time step $t$ is denoted as $Q$<sub>t</sub>($a$).
    
We want $Q$<sub>t</sub>($a$) to be close to $q$<sub>*</sub>($a$). If we keep estimates of the action values, then at each time step there is an action with the greatest expected value. If we choose the action with that value, these types of actions are considered greedy actions. Choosing one a greedy action means you are exploiting your current knowledge of action values. 
    
Nongreedy actions are said to be exploring actions, because we are improving our estimate of the nongreedy action's value. 
    
"Exploitation is the right thing to do to maximize the expected reward on the one step, but exploration may produce the greater reward in the long run."
 
Since we can't possibly explore and exploit with one action, there's a balance to be struck between exploration and exploitation.    

In each specific case, if it is better to explore or exploit depends on a multitude of factors; the values of the estimates, uncertainties, and the number of steps left. Lots of different, sophisticated, methods exist to balance between exploration and exploitation for certain formulations of the k-armed bandit problem. Issue is, most of these methods make "strong assumptions about stationary and prior knowledge that are either violated or impossible to verify in applications and in the full reinforcement learning problem that we consider in subsequent chapters."
    
In chapter 2 we will cover several simple balancing methods for the k-armed bandit problem and we will see that they are superior to methods that always exploit. Balancing exploration and exploitation is a "distinctive challenge" in RL; our simple version of the k-armed bandit problem will allow us to show this in a very clear way.

### 2.2 Action-Value Methods

A natural way to estimate the true value of an action is to average the rewards received:

$Q$<sub>$t$</sub>($a$) = $\frac{\sum_{i=1}^{t - 1}R_i * 1}{\sum_{i=1}^{t-1} 1}$

Where $1$ is the same as $1_{predicate}$ in the textbook.

From text: the 1 in the formula above should be *1<sub>A<sub>i</sub>=a</sub>*

$1_{predicate}$ denotes a random variable that is 1 if $predicate$ is true and 0 if it isn't. If the denominator becomes zero, then we define $Q_t(a)$ by some other value, like 0. As the denominator grows towards infinity, by the law of large numbers, $Q_t(a)$ converges towards $q_*(a)$

This is called the sample-average method; estimates action values because each estimate is average of the sample of relevant rewards. 

Can write a greedy action selection method as: $A_t = argmax  (Q_t(a))$. Greedy action selection *always* exploits current knowledge to maximize the immediate reward. It does not try at all to sample inferior actions and discover if they might actually be better.

An easy way to most often behave greedily, but then with a small probability, call it $\varepsilon$, randomly sample from all of the moves with equal probability. Ignore the action-value estimates. Methods like this are called $\varepsilon-greedy$ methods. With these methods, as the number of steps increases, every action will be sampled an infinite number of times, guaranteeing that all the $Q_t(a)$ converge to $q_*(a)$. This implies that the probability of selecting the optimal action converges to more than $1 - \varepsilon$, which is near certainty. 

### 2.3 The 10-armed Testbed

To assess the effectiveness of the greedy and $\varepsilon - greedy$ methods, the book authors compared them numerically on a bunch of test problems. It was a set of 2000 randomly generated k-armed bandit problems where k = 10. For each bandit problem, the action values $q_*(a)$; a = 1, 2, 3, ..., 10 were sampled from a normal distribution with mean 0 and variance 1. When a learning method applied to a problem picked action $A_t$, the reward $R_t$ was sampled from a normal distribution with mean $q_*(A_t)$ and variance 1. They call this set of tasks the 10 armed testbed.

For any learning method, they can measure its performance and behavior as it learns and gains experience over 1000 time steps on a particular bandit problem. This makes up one run. 

They repeated this for 2000 runs, each with a different bandit problem, and so obtained measures of the learning algorithm's average behavior.

The plot in the textbook, figure 2.2, shows that a greedy-only method will most quickly maximize reward in the short term, but in the long term it will get less reward than a $\varepsilon$ greedy method. The method with $\varepsilon = 0.1$ rapidly maximized reward, and continued to maximize reward over the long run. When $\varepsilon$ is 0.01, it more slowly maximizes reward but if it is allowed to run for long enough, it will eventually perform better than when $\varepsilon = 0.1$. A second plot shows that a greedy-only method only learns to make the optimal move about a third of the time, while the $\varepsilon = 0.1$ method learns to make the optimal move about 85% of the time. Again, $\varepsilon=0.01$ is slower to converge, but given enough time it will outperform $\varepsilon=0.1$.

### 2.4 Incremental implementation

Action-value methods covered so far all estimate values as sample averages of the observed rewards. Now we look at how these averages can be computed efficiently, with constant memory and constant per-time-step computation.

To help make notation simpler, we look at a single action. Have $R_i$ denote the reward received after the $i$th selection of this action and let $Q_n$ be the estimate of the action value after it has been picked $n-1$ times.

We can now write the $Q_n$ as:

$Q_n \stackrel{.}{=} \frac{R_1 + R_2 + \dotsb + R_n}{n-1}$

An obvious implementation would be to have an array keeping a record of all the rewards and then perform the above computation whenever the estimated value of the action was needed. But, if we do it this way, then memory and computational requirements would grow over time as more reward is seen. Each additional reward would need more memory to store it and it would add more time to the calculation.

It isn't really necessary to do it this way. We can come up with incremental formulas to update the averages with small, constant amount of computation required to process each reward.

Given $Q_n$ and the nth reward $R_n$, the new average of all n rewards can be found with:

$Q_{n+1} = \frac{1}{n}\sum_{i=1}^{n}R_i$

$Q_{n+1} = \frac{1}{n}(R_n + \sum_{i=1}^{n-1}R_i)$

$Q_{n+1} = \frac{1}{n}(R_n + (n-1)\frac{1}{n-1}\sum_{i=1}^{n}R_i)$

$Q_{n+1} = \frac{1}{n}(R_n + (n-1)Q_n)$

$Q_{n+1} = \frac{1}{n}(R_n + nQ_n - Q_n)$

$Q_{n+1} = Q_n + \frac{1}{n}[R_n - Q_n]$

This holds even for n = 1, giving $Q_2 = R_1$ for arbitrary $Q_1$. It requires memory for only $Q_n$ and $n$.